In [1]:
#establishing environment
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import scipy
from scipy.stats import pearsonr, ttest_rel, ttest_ind
from scipy import signal as sig
import matplotlib.patches as mpatches
from scipy.io import loadmat, savemat
from ieeg.auth import Session

In [2]:
#loads all the cat spikes before they are in the GDF
#NOT NECESSARY TO LOAD AFTER GDFS ARE CREATED

catspikes = loadmat('/Users/carlosaguila/PycharmProjects/CNT_Interictal_Spikes/Cat/Bink_files/catspikes.mat')
numSpikes = catspikes['numSpikes'][0] #spike count per event
spikeStart = catspikes['spikeStart'][0] #array of where the spike starts
spikeStop = catspikes['spikeStop'][0] #spike ends
evStart = catspikes['evStart'][0] #event starts
evStop = catspikes['evStop'][0] #event stops

In [5]:
seizure_leaders_C1 = []
seizure_leaders_C2 = []
seizure_leaders_C3 = []
seizure_leaders_C4 = []
seizure_leaders_C5 = []

for i in range(len(numSpikes)):
    for j in range(len(numSpikes[i][0])):
        for k in range(len(numSpikes[i][0][j][0])):
            if numSpikes[i][0][j][0][k] >= 20:
                if i == 0:
                    seizure_leaders_C1.append(np.array([j,k,spikeStart[i][0][j][0][k][0][0], evStart[i][0][j][0][k]]))
                if i == 1:
                    seizure_leaders_C2.append(np.array([j,k,spikeStart[i][0][j][0][k][0][0], evStart[i][0][j][0][k]]))
                if i == 2:
                    seizure_leaders_C3.append(np.array([j,k,spikeStart[i][0][j][0][k][0][0], evStart[i][0][j][0][k]]))
                if i == 3:
                    seizure_leaders_C4.append(np.array([j,k,spikeStart[i][0][j][0][k][0][0], evStart[i][0][j][0][k]]))
                if i == 4:
                    seizure_leaders_C5.append(np.array([j,k,spikeStart[i][0][j][0][k][0][0], evStart[i][0][j][0][k]]))

In [210]:
def find_sz_start(CH,cat_experiment):
    sz_start_detectch = []
    for array in cat_experiment:
        if array[0] == CH:
            sz_start_detectch.append([array[2]])
    return np.array(sz_start_detectch)

In [227]:
cats = [seizure_leaders_C1,seizure_leaders_C2,seizure_leaders_C3,seizure_leaders_C4,seizure_leaders_C5]
SOZ = [3,2,2,2,5]
elec = [range(48),range(48),range(64),range(64),range(64)]
concat_sz_C1 = [i[2] for i in seizure_leaders_C1]
fs = 2000
min_2 = fs*3*60;

In [228]:
#here we will try to find time values underneath the reference one.

sz_start_detectch_C1 = find_sz_start(3,seizure_leaders_C1) #gives me a list with seizure points captured on electrode 4 (SOZ)
idx_possible_sz = []


for i in np.arange(0,len(sz_start_detectch_C1),1):
    temp = []
    find = np.where((concat_sz_C1 > (sz_start_detectch_C1[i]-min_2)) & (concat_sz_C1 < sz_start_detectch_C1[i]))[0]
    for j in find:
        temp.append(concat_sz_C1[j])
    temp2 = np.min(temp)        
    idx_possible_sz.append([find,temp2,np.where(temp == temp2)[0]])

In [229]:
poss_idxs = []
for i in idx_possible_sz:
    poss_idxs.append(i[0][i[2]])

In [230]:
print(idx_possible_sz[3])

[array([  0,   1,   2,   3,  56,  57,  58,  59, 113, 114, 115, 169, 170,
       171, 225, 226, 227, 228, 266, 267, 268, 269, 321, 322, 323, 377,
       378, 379, 433, 434, 435, 465, 518, 519, 520]), 1988988, array([0])]


In [231]:
print(poss_idxs)

[array([112]), array([112]), array([0]), array([0]), array([0]), array([170]), array([170]), array([2]), array([3]), array([60]), array([174]), array([6]), array([7]), array([7]), array([942]), array([ 943, 1025]), array([179, 276]), array([1026, 1070]), array([629]), array([1027, 1071]), array([279]), array([1029, 1073]), array([631]), array([1075]), array([1032, 1076]), array([1033, 1077]), array([  20, 1034]), array([632]), array([633]), array([1036]), array([1037]), array([1039, 1083]), array([1040, 1084]), array([591]), array([1042, 1086]), array([253]), array([1044]), array([635]), array([1046, 1090]), array([1047, 1091]), array([1048, 1092]), array([1049, 1093]), array([1050, 1094]), array([1051, 1095]), array([1052]), array([636]), array([1054, 1098]), array([1055, 1099, 1135]), array([1154]), array([1057, 1101]), array([1156]), array([1103]), array([1060, 1104]), array([1105]), array([1062, 1106, 1142]), array([1063, 1107, 1143])]


In [237]:
electrodes_in_q_int = []
elec_in_q = []
for idxs in poss_idxs:
    for i in idxs:
        electrodes_in_q_int.append(seizure_leaders_C1[i])

for array in electrodes_in_q_int:
    elec_in_q.append(array[0])
    
def remove_depth(elec_in_q):
    elecs = [x for x in elec_in_q if (x<33)]
    return elecs

print(elec_in_q)
print(remove_depth(elec_in_q))

[2, 2, 0, 0, 0, 3, 3, 0, 0, 1, 3, 0, 0, 0, 41, 41, 43, 3, 5, 43, 44, 32, 43, 44, 5, 43, 44, 32, 44, 43, 44, 43, 44, 0, 43, 32, 32, 43, 43, 43, 44, 43, 44, 15, 43, 44, 4, 43, 32, 43, 44, 43, 44, 43, 44, 43, 44, 43, 44, 43, 44, 43, 32, 43, 44, 43, 44, 46, 47, 43, 44, 47, 44, 43, 44, 44, 43, 44, 46, 43, 44, 46]
[2, 2, 0, 0, 0, 3, 3, 0, 0, 1, 3, 0, 0, 0, 3, 5, 32, 5, 32, 0, 32, 32, 15, 4, 32, 32]


def most_frequent(List):
    counter = 0
    num = List[0]
     
    for i in List:
        curr_frequency = List.count(i)
        if(curr_frequency> counter):
            counter = curr_frequency
            num = i
 
    return num

print(most_frequent(elec_in_q))
print(most_frequent(remove_depth(elec_in_q)))

----

In [259]:
#Same thing but with cat 2

def find_SOZ_SZ_referenced(seizure_leaders, detectCH):
    concat_sz_C1 = [i[2] for i in seizure_leaders]
    sz_start_detectch_C1 = find_sz_start(detectCH,seizure_leaders) #gives me a list with seizure points captured on electrode 4 (SOZ)
    idx_possible_sz = []


    for i in np.arange(0,len(sz_start_detectch_C1),1):
        temp = []
        find = np.where((concat_sz_C1 > (sz_start_detectch_C1[i]-min_2)) & (concat_sz_C1 < sz_start_detectch_C1[i]))[0]
        for j in find:
            temp.append(concat_sz_C1[j])
        if temp != []:   
            temp2 = np.min(temp)        
            idx_possible_sz.append([find,temp2,np.where(temp == temp2)[0]])

    poss_idxs = []
    for i in idx_possible_sz:
        poss_idxs.append(i[0][i[2]])

    electrodes_in_q_int = []
    elec_in_q = []
    for idxs in poss_idxs:
        for i in idxs:
            electrodes_in_q_int.append(seizure_leaders[i])

    for array in electrodes_in_q_int:
        elec_in_q.append(array[0])

    def remove_depth(elec_in_q):
        elecs = [x for x in elec_in_q if (x<33)]
        return elecs
    
    return print(elec_in_q), print(remove_depth(elec_in_q)), print(most_frequent(elec_in_q)), print(most_frequent(remove_depth(elec_in_q)))

In [260]:
find_SOZ_SZ_referenced(seizure_leaders_C2, 2)

[44, 46, 47, 39, 38, 38, 2, 1, 2, 15, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 42, 40, 45, 47, 1, 1, 2, 2, 44, 45, 1, 2, 6, 46, 47, 42, 2, 1, 14, 39, 4, 2, 2, 40, 2, 2, 2, 2, 38, 2, 14, 2, 2, 2, 1, 2, 39, 11, 2, 2, 39, 39, 19, 2, 43, 2, 2, 2, 2, 44, 44, 45, 43, 44, 45, 46, 44, 45, 46, 44, 45, 44, 45, 46, 43, 44, 45, 45, 44, 45, 46, 44, 45, 46, 42, 44, 45, 47, 42, 7, 45, 43, 45, 44, 45, 47, 44, 45, 47, 45, 46, 45, 44, 45, 46, 7, 44, 45, 46, 47, 45, 43, 44, 45, 46, 47, 42, 44, 45, 46, 47, 45, 47, 44, 45, 46, 47, 44, 45, 36, 36, 36, 45, 47, 42, 45, 46, 47, 45, 47, 43, 44, 45, 46, 46, 44, 45, 46, 47, 42]
[2, 1, 2, 15, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 1, 2, 6, 2, 1, 14, 4, 2, 2, 2, 2, 2, 2, 2, 14, 2, 2, 2, 1, 2, 11, 2, 2, 19, 2, 2, 2, 2, 2, 7, 7]
2
2


(None, None, None, None)

In [261]:
find_SOZ_SZ_referenced(seizure_leaders_C3, 2)

[14, 38, 29, 29, 24, 24, 5, 6, 54, 24]
[14, 29, 29, 24, 24, 5, 6, 24]
24
24


(None, None, None, None)

In [262]:
find_SOZ_SZ_referenced(seizure_leaders_C4, 2)

[42, 41, 0, 0, 40, 33, 38, 39, 38, 39, 44, 44, 38, 37, 38, 39, 0, 38, 18, 39, 45, 18, 37, 38, 39, 38, 5, 14, 14, 43, 38, 37, 45, 43, 37, 38, 45, 47, 40, 9, 34, 38, 38, 17, 34, 35, 39, 43, 38, 39, 45, 46, 47, 37, 38, 38, 33, 38, 39, 44, 38, 44, 45, 38, 37, 37, 45, 38, 36, 18, 43, 44, 37, 39, 37, 37, 33, 14, 43, 37, 38, 39, 39, 38, 38, 39, 1, 35, 37, 10, 14, 45, 14, 6, 39, 38, 37, 43, 46]
[0, 0, 0, 18, 18, 5, 14, 14, 9, 17, 18, 14, 1, 10, 14, 14, 6]
38
14


(None, None, None, None)

In [263]:
find_SOZ_SZ_referenced(seizure_leaders_C5, 5)

[9, 9, 9, 63, 39, 9, 44, 44, 9, 0, 0, 27, 29, 35, 36, 37, 35, 36, 37, 38, 39]
[9, 9, 9, 9, 9, 0, 0, 27, 29]
9
9


(None, None, None, None)

----

Now - we will equate the referenced channel to the channel with MOST seizures and see if it changes.

In [268]:
#find electrode with most seizures
#we would want to count most_frequent but for electrodes in the seizure_leaders

concat_elec_C1 = [i[0] for i in seizure_leaders_C1]
concat_elec_C2 = [i[0] for i in seizure_leaders_C2]
concat_elec_C3 = [i[0] for i in seizure_leaders_C3]
concat_elec_C4 = [i[0] for i in seizure_leaders_C4]
concat_elec_C5 = [i[0] for i in seizure_leaders_C5]

max_sz_elec = [most_frequent(concat_elec_C1),most_frequent(concat_elec_C2),most_frequent(concat_elec_C3),most_frequent(concat_elec_C4),most_frequent(concat_elec_C5)] 
max_sz_elec_noDE = [most_frequent(remove_depth(concat_elec_C1)),most_frequent(remove_depth(concat_elec_C2)),most_frequent(remove_depth(concat_elec_C3)),most_frequent(remove_depth(concat_elec_C4)),most_frequent(remove_depth(concat_elec_C5))]

In [269]:
print(max_sz_elec)
print(max_sz_elec_noDE)

[2, 2, 38, 45, 9]
[2, 2, 6, 32, 9]


In [270]:
find_SOZ_SZ_referenced(seizure_leaders_C1, max_sz_elec_noDE[0])

[2, 2, 0, 0, 0, 3, 3, 0, 0, 1, 3, 0, 0, 0, 41, 41, 43, 3, 5, 43, 44, 32, 43, 44, 5, 43, 44, 32, 44, 43, 44, 43, 44, 0, 43, 32, 32, 43, 43, 43, 44, 43, 44, 15, 43, 44, 4, 43, 32, 43, 44, 43, 44, 43, 44, 43, 44, 43, 44, 43, 44, 43, 32, 43, 44, 43, 44, 46, 47, 43, 44, 47, 44, 43, 44, 44, 43, 44, 46, 43, 44, 46]
[2, 2, 0, 0, 0, 3, 3, 0, 0, 1, 3, 0, 0, 0, 3, 5, 32, 5, 32, 0, 32, 32, 15, 4, 32, 32]
43
0


(None, None, None, None)

In [271]:
find_SOZ_SZ_referenced(seizure_leaders_C2, max_sz_elec_noDE[1])

[44, 46, 47, 39, 38, 38, 2, 1, 2, 15, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 42, 40, 45, 47, 1, 1, 2, 2, 44, 45, 1, 2, 6, 46, 47, 42, 2, 1, 14, 39, 4, 2, 2, 40, 2, 2, 2, 2, 38, 2, 14, 2, 2, 2, 1, 2, 39, 11, 2, 2, 39, 39, 19, 2, 43, 2, 2, 2, 2, 44, 44, 45, 43, 44, 45, 46, 44, 45, 46, 44, 45, 44, 45, 46, 43, 44, 45, 45, 44, 45, 46, 44, 45, 46, 42, 44, 45, 47, 42, 7, 45, 43, 45, 44, 45, 47, 44, 45, 47, 45, 46, 45, 44, 45, 46, 7, 44, 45, 46, 47, 45, 43, 44, 45, 46, 47, 42, 44, 45, 46, 47, 45, 47, 44, 45, 46, 47, 44, 45, 36, 36, 36, 45, 47, 42, 45, 46, 47, 45, 47, 43, 44, 45, 46, 46, 44, 45, 46, 47, 42]
[2, 1, 2, 15, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 1, 2, 6, 2, 1, 14, 4, 2, 2, 2, 2, 2, 2, 2, 14, 2, 2, 2, 1, 2, 11, 2, 2, 19, 2, 2, 2, 2, 2, 7, 7]
2
2


(None, None, None, None)

In [272]:
find_SOZ_SZ_referenced(seizure_leaders_C3, max_sz_elec_noDE[2])

[14, 38, 29, 29, 24, 25, 26, 60, 6, 8, 54, 24]
[14, 29, 29, 24, 25, 26, 6, 8, 24]
29
29


(None, None, None, None)

In [273]:
find_SOZ_SZ_referenced(seizure_leaders_C4, max_sz_elec_noDE[3])

[44, 0, 37, 38, 0, 33, 38, 39, 44, 44, 37, 38, 39, 0, 38, 40, 0, 14, 39, 45, 40, 37, 38, 39, 38, 5, 14, 14, 43, 38, 33, 5, 38, 37, 45, 43, 37, 38, 45, 47, 43, 36, 38, 38, 39, 44, 10, 40, 9, 34, 38, 38, 17, 34, 35, 39, 43, 38, 39, 45, 46, 47, 18, 37, 38, 37, 38, 37, 37, 38, 38, 33, 38, 39, 44, 38, 34, 44, 45, 38, 33, 38, 39, 37, 37, 37, 38, 45, 38, 38, 33, 18, 46, 37, 39, 46, 37, 37, 33, 14, 19, 37, 38, 39, 1, 39, 14, 38, 15, 38, 21, 33, 4, 43, 0, 2, 2, 0, 45, 15, 43, 35, 38, 37, 43, 45, 47]
[0, 0, 0, 0, 14, 5, 14, 14, 5, 10, 9, 17, 18, 18, 14, 19, 1, 14, 15, 21, 4, 0, 2, 2, 0, 15]
38
0


(None, None, None, None)

In [275]:
find_SOZ_SZ_referenced(seizure_leaders_C5, max_sz_elec_noDE[4])

[13, 13, 9, 41, 41, 13, 9, 44, 46, 39, 9, 55, 44, 44, 44, 9, 0, 0, 27, 29, 29, 35, 36, 37, 38, 39]
[13, 13, 9, 13, 9, 9, 9, 0, 0, 27, 29, 29]
9
9


(None, None, None, None)

----

In [286]:
#export mat files to run in the sequence code - create gdf

gdf_c1 = []
gdf_c2 = []
gdf_c3 = []
gdf_c4 = []
gdf_c5 = []

for i in range(len(numSpikes)):
    for j in range(len(numSpikes[i][0])):
        for k in range(len(numSpikes[i][0][j][0])):
            if numSpikes[i][0][j][0][k] >= 20:
                if i == 0:
                    gdf_c1.append(np.array([j,spikeStart[i][0][j][0][k][0][0]]))
                if i == 1:
                    gdf_c2.append(np.array([j,spikeStart[i][0][j][0][k][0][0]]))
                if i == 2:
                    gdf_c3.append(np.array([j,spikeStart[i][0][j][0][k][0][0]]))
                if i == 3:
                    gdf_c4.append(np.array([j,spikeStart[i][0][j][0][k][0][0]]))
                if i == 4:
                    gdf_c5.append(np.array([j,spikeStart[i][0][j][0][k][0][0]]))
                    
gdf_c1 = np.vstack(gdf_c1)
gdf_c2 = np.vstack(gdf_c2)
gdf_c3 = np.vstack(gdf_c3)
gdf_c4 = np.vstack(gdf_c4)
gdf_c5 = np.vstack(gdf_c5)

In [287]:
from scipy.io import loadmat, savemat

gdf_array = np.zeros((5,1), dtype=object)
gdf_array[0,0] = gdf_c1
gdf_array[1,0] = gdf_c2
gdf_array[2,0] = gdf_c3
gdf_array[3,0] = gdf_c4
gdf_array[4,0] = gdf_c5

In [288]:
savemat('/Users/carlosaguila/Downloads/gdf_array_cats.mat', {'gdf_array_cats':gdf_array})

---

In [327]:
#import

sequences = loadmat('/Users/carlosaguila/PycharmProjects/CNT_Interictal_Spikes/Cat/sequences.mat')

In [328]:
seqs_c1 = sequences['seqs_c1'][0]
seqs_c2 = sequences['seqs_c2'][0]
seqs_c3 = sequences['seqs_c3'][0]
seqs_c4 = sequences['seqs_c4'][0]
seqs_c5 = sequences['seqs_c5'][0]


In [329]:
def onset_seqs(sequence):
    onset_seqs = []
    for seqs in sequence:
        onset_seqs.append(seqs[0])
    onset_seqs = np.vstack(onset_seqs)
    return onset_seqs

In [330]:
onset_sz_c1 = onset_seqs(seqs_c1)
onset_sz_c2 = onset_seqs(seqs_c2)
onset_sz_c3 = onset_seqs(seqs_c3)
onset_sz_c4 = onset_seqs(seqs_c4)
onset_sz_c5 = onset_seqs(seqs_c5)

In [331]:
onset_elec_c1 = [x[0] for x in onset_sz_c1]
onset_elec_c2 = [x[0] for x in onset_sz_c2]
onset_elec_c3 = [x[0] for x in onset_sz_c3]
onset_elec_c4 = [x[0] for x in onset_sz_c4]
onset_elec_c5 = [x[0] for x in onset_sz_c5]

In [332]:
#c1
print(most_frequent(onset_elec_c1)), print(most_frequent(remove_depth(onset_elec_c1)))

44
1


(None, None)

In [333]:
#c2
print(most_frequent(onset_elec_c2)), print(most_frequent(remove_depth(onset_elec_c2)))

3
3


(None, None)

In [334]:
#c3
print(most_frequent(onset_elec_c3)), print(most_frequent(remove_depth(onset_elec_c3)))

42
10


(None, None)

In [335]:
#c4
print(most_frequent(onset_elec_c4)), print(most_frequent(remove_depth(onset_elec_c4)))

39
15


(None, None)

In [336]:
#c5
print(most_frequent(onset_elec_c5)), print(most_frequent(remove_depth(onset_elec_c5)))

10
10


(None, None)